# Macbeth meets RNN
We'll be using the story "Macbeth" by William Shakeaspeare.
The story is in public domain; the text file was obtained from nltk.

In [1]:
import nltk
import spacy
from spacy import displacy
import random
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from pickle import dump,load

Using TensorFlow backend.


## Functions

In [2]:
# desconsider the punctuation
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) \
            if token.text not in ' \n\n   \n \n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

# create the LSTM model
def create_model(vocab_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocab_size,seq_len,input_length = seq_len))
    model.add(LSTM(seq_len*2,return_sequences=True))
    model.add(LSTM(seq_len*2))
    model.add(Dense(50,activation='relu'))
    model.add(Dense(vocab_size,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    return model

# the function takes in the 'model' the tokenizer which has knowledge about the vocabulary in what ID number goes
# with what word. Some seed text you want to start off with. And this is robust enough to have shorter seed text
# or longer seed text and sequence length.

# for better results, you should make the text the same length as what was trained on. Otherwise, you have to pad it
# and then the number of words we want to generate.

def generator_text(model, tokenizer,seq_len, seed_text,num_gen_words):
    
    # the output text which is the final output
    output_text = []
    input_text = seed_text # 25 words
    
    for i in range(num_gen_words):
        # take the input text string and encode it to be a sequence
        # we transform those raw text data into sequences of numbers.
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # then if the seed text is too short or to long I may need to pad it (cut off or add it)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating = 'pre')
        # after that I'm going to predict the class probabilities for each word.
        # the line below is essentially going to throw out the entire vocabulary.
        # Assign a probability to the most likely next word.
        pred_word_ind = model.predict_classes(pad_encoded, verbose = 0)[0] #[0] return the index of that particular word
        pred_word = tokenizer.index_word[pred_word_ind]
        # then we're going to take in the input text and I'm going to add the predicted word
        input_text += ' ' + pred_word
        output_text.append(pred_word)
    
    return ' '.join(output_text)

## Data Preprocessing

In [3]:
# import the "Macbeth" by William Shakeaspeare
path = nltk.data.find('corpora/gutenberg/shakespeare-macbeth.txt')
macbeth = open(path, 'r').read()

In [4]:
# setting the basics configs
nlp = spacy.load('en_core_web_sm', disable=['parser','tagger','ner'])
nlp.max_length = 1198623
doc = nlp(macbeth)
tokens = separate_punc(str(doc))

In [5]:
len(tokens)

18089

In [6]:
# pass 25 words --> network predict 26th
train_len = 25+1
text_sequences = []
for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

In [7]:
' '.join(text_sequences[0])

'the tragedie of macbeth by william shakespeare 1603 actus primus scoena prima thunder and lightning enter three witches 1 when shall we three meet againe in'

In [8]:
# So what we`ve done here is we`ve essentially replaced our original text sequences which are just sequences
# of text (26 long) to sequences that happen to have numbers in place of those words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

In [9]:
tokenizer.index_word

{1: 'the',
 2: 'and',
 3: 'to',
 4: 'of',
 5: 'i',
 6: 'a',
 7: 'that',
 8: 'my',
 9: 'you',
 10: 'in',
 11: 'not',
 12: 'is',
 13: 'it',
 14: 'with',
 15: 'his',
 16: 'be',
 17: 'macb',
 18: "'s",
 19: 'your',
 20: 'our',
 21: 'haue',
 22: 'but',
 23: 'what',
 24: "'",
 25: 'me',
 26: 'he',
 27: 'for',
 28: 'this',
 29: 'all',
 30: 'so',
 31: 'him',
 32: 'as',
 33: 'thou',
 34: 'we',
 35: 'enter',
 36: 'which',
 37: 'are',
 38: 'will',
 39: 'they',
 40: 'shall',
 41: 'no',
 42: 'then',
 43: 'do',
 44: 'their',
 45: 'thee',
 46: 'macbeth',
 47: 'vpon',
 48: 'on',
 49: 'macd',
 50: 'from',
 51: 'yet',
 52: 'thy',
 53: 'king',
 54: 'come',
 55: 'vs',
 56: 'there',
 57: 'now',
 58: 'hath',
 59: 'at',
 60: 'more',
 61: 'who',
 62: 'good',
 63: 'rosse',
 64: 'by',
 65: 'them',
 66: 'lady',
 67: 'would',
 68: 'time',
 69: 'can',
 70: 'was',
 71: 'like',
 72: 'her',
 73: 'if',
 74: 'let',
 75: 'should',
 76: 'did',
 77: 'when',
 78: 'where',
 79: 'say',
 80: 'were',
 81: 'make',
 82: 'banquo'

In [10]:
for i in sequences[0]:
    print(f"{i} : {tokenizer.index_word[i]}")

1 : the
3526 : tragedie
4 : of
46 : macbeth
64 : by
3525 : william
3524 : shakespeare
3523 : 1603
505 : actus
3522 : primus
3521 : scoena
504 : prima
294 : thunder
2 : and
1376 : lightning
35 : enter
213 : three
331 : witches
100 : 1
77 : when
40 : shall
34 : we
213 : three
359 : meet
136 : againe
10 : in


In [11]:
# it also has things like word counts which essentially counts how many times these words show up
tokenizer.word_counts

OrderedDict([('the', 16774),
             ('tragedie', 4),
             ('of', 8740),
             ('macbeth', 1564),
             ('by', 1253),
             ('william', 6),
             ('shakespeare', 7),
             ('1603', 8),
             ('actus', 113),
             ('primus', 10),
             ('scoena', 11),
             ('prima', 116),
             ('thunder', 221),
             ('and', 14152),
             ('lightning', 41),
             ('enter', 2096),
             ('three', 326),
             ('witches', 200),
             ('1', 825),
             ('when', 1034),
             ('shall', 1763),
             ('we', 2143),
             ('meet', 180),
             ('againe', 545),
             ('in', 5200),
             ('or', 962),
             ('raine', 26),
             ('2', 520),
             ('hurley', 26),
             ('burley', 26),
             ("'s", 3406),
             ('done', 910),
             ('battaile', 26),
             ('lost', 156),
             ('wonne',

In [12]:
vocab_size = len(tokenizer.word_counts)
vocab_size

3527

In [13]:
sequences = np.array(sequences)
sequences

array([[   1, 3526,    4, ...,  359,  136,   10],
       [3526,    4,   46, ...,  136,   10,  294],
       [   4,   46,   64, ...,   10,  294, 1376],
       ...,
       [   2,  224,   30, ..., 1379, 3527,    1],
       [ 224,   30,  503, ..., 3527,    1, 3526],
       [  30,  503,    3, ...,    1, 3526,    4]])

In [14]:
# grab everthing excepts the last column
X = sequences[:,:-1]
# grab the last column
y = sequences[:,-1]
# the plus 1 is because the way Keras padding works. It needs an extra one to holds zero
y = to_categorical(y,num_classes = vocab_size+1)
seq_len = X.shape[1]
X.shape
# 18711 sequences and so those were essentially those shifted 25 words sentences and then how many words are in each sentence

(18063, 25)

## Build the Neural Network

In [15]:
model = create_model(vocab_size+1,seq_len)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            88200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 50)            15200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 3528)              179928    
Total params: 306,078
Trainable params: 306,078
Non-trainable params: 0
_________________________________________________________________


In [16]:
# previously I was using the terms train test split it wasn't really a train split
# it was actually features labels split. Because there's nothing to test
# There's no right answer as far as what text generated should look like.
# instead we are really just texting these features against predicted label

# the batch_size is how many sequences you want to passen a time.
model.fit(X,y,batch_size=128,epochs=390,verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/390
18063/18063 [==============================] - ETA: 6:33 - loss: 8.1685 - acc: 0.0000e+0 - ETA: 3:27 - loss: 8.1682 - acc: 0.0039    - ETA: 2:24 - loss: 8.1677 - acc: 0.005 - ETA: 1:52 - loss: 8.1674 - acc: 0.003 - ETA: 1:33 - loss: 8.1670 - acc: 0.007 - ETA: 1:22 - loss: 8.1666 - acc: 0.007 - ETA: 1:14 - loss: 8.1660 - acc: 0.010 - ETA: 1:06 - loss: 8.1656 - acc: 0.008 - ETA: 1:01 - loss: 8.1651 - acc: 0.008 - ETA: 56s - loss: 8.1645 - acc: 0.010 - ETA: 53s - loss: 8.1640 - acc: 0.00 - ETA: 49s - loss: 8.1630 - acc: 0.01 - ETA: 47s - loss: 8.1622 - acc: 0.01 - ETA: 45s - loss: 8.1610 - acc: 0.01 - ETA: 43s - loss: 8.1601 - acc: 0.01 - ETA: 42s - loss: 8.1588 - acc: 0.01 - ETA: 40s - loss: 8.1577 - acc: 0.01 - ETA: 39s - loss: 8.1555 - acc: 0.01 - ETA: 37s - loss: 8.1536 - acc: 0.01 - ETA: 36s - loss: 8.1512 - acc: 0.01 - ETA: 35s - loss: 8.1472 - acc: 0.01 - ETA: 34s - loss: 8.1433 - acc: 0.01 - ETA: 33s - loss: 8.1383 - acc:

18063/18063 [==============================] - ETA: 18s - loss: 6.6851 - acc: 0.03 - ETA: 23s - loss: 6.5941 - acc: 0.03 - ETA: 22s - loss: 6.5305 - acc: 0.04 - ETA: 22s - loss: 6.5563 - acc: 0.03 - ETA: 22s - loss: 6.5869 - acc: 0.04 - ETA: 21s - loss: 6.6233 - acc: 0.03 - ETA: 21s - loss: 6.6206 - acc: 0.03 - ETA: 20s - loss: 6.6193 - acc: 0.03 - ETA: 20s - loss: 6.6202 - acc: 0.03 - ETA: 20s - loss: 6.6431 - acc: 0.03 - ETA: 20s - loss: 6.6434 - acc: 0.03 - ETA: 20s - loss: 6.6159 - acc: 0.03 - ETA: 20s - loss: 6.5909 - acc: 0.03 - ETA: 20s - loss: 6.5706 - acc: 0.03 - ETA: 19s - loss: 6.5781 - acc: 0.03 - ETA: 19s - loss: 6.5824 - acc: 0.03 - ETA: 19s - loss: 6.5870 - acc: 0.03 - ETA: 19s - loss: 6.5922 - acc: 0.03 - ETA: 18s - loss: 6.5774 - acc: 0.03 - ETA: 19s - loss: 6.5661 - acc: 0.03 - ETA: 18s - loss: 6.5554 - acc: 0.03 - ETA: 18s - loss: 6.5673 - acc: 0.03 - ETA: 18s - loss: 6.5826 - acc: 0.03 - ETA: 18s - loss: 6.5797 - acc: 0.03 - ETA: 18s - loss: 6.5803 - acc: 0.03 - ETA

18063/18063 [==============================] - ETA: 17s - loss: 6.2144 - acc: 0.06 - ETA: 19s - loss: 6.2971 - acc: 0.05 - ETA: 18s - loss: 6.2785 - acc: 0.04 - ETA: 18s - loss: 6.3241 - acc: 0.04 - ETA: 17s - loss: 6.3451 - acc: 0.04 - ETA: 17s - loss: 6.3636 - acc: 0.04 - ETA: 18s - loss: 6.3947 - acc: 0.04 - ETA: 18s - loss: 6.3833 - acc: 0.03 - ETA: 18s - loss: 6.4043 - acc: 0.03 - ETA: 18s - loss: 6.3877 - acc: 0.03 - ETA: 18s - loss: 6.3770 - acc: 0.03 - ETA: 18s - loss: 6.3732 - acc: 0.03 - ETA: 18s - loss: 6.3643 - acc: 0.03 - ETA: 17s - loss: 6.3549 - acc: 0.03 - ETA: 18s - loss: 6.3555 - acc: 0.03 - ETA: 18s - loss: 6.3592 - acc: 0.03 - ETA: 18s - loss: 6.3699 - acc: 0.03 - ETA: 18s - loss: 6.3760 - acc: 0.03 - ETA: 17s - loss: 6.3671 - acc: 0.03 - ETA: 18s - loss: 6.3615 - acc: 0.03 - ETA: 17s - loss: 6.3655 - acc: 0.03 - ETA: 17s - loss: 6.3676 - acc: 0.03 - ETA: 17s - loss: 6.3712 - acc: 0.03 - ETA: 17s - loss: 6.3799 - acc: 0.03 - ETA: 17s - loss: 6.3801 - acc: 0.03 - ETA

18063/18063 [==============================] - ETA: 16s - loss: 6.4665 - acc: 0.03 - ETA: 18s - loss: 6.1823 - acc: 0.03 - ETA: 20s - loss: 6.1854 - acc: 0.03 - ETA: 20s - loss: 6.1790 - acc: 0.02 - ETA: 20s - loss: 6.2312 - acc: 0.02 - ETA: 19s - loss: 6.2026 - acc: 0.02 - ETA: 19s - loss: 6.2163 - acc: 0.03 - ETA: 18s - loss: 6.2389 - acc: 0.03 - ETA: 18s - loss: 6.2359 - acc: 0.03 - ETA: 18s - loss: 6.2190 - acc: 0.03 - ETA: 18s - loss: 6.2190 - acc: 0.03 - ETA: 18s - loss: 6.2372 - acc: 0.03 - ETA: 18s - loss: 6.2512 - acc: 0.03 - ETA: 18s - loss: 6.2560 - acc: 0.03 - ETA: 17s - loss: 6.2533 - acc: 0.03 - ETA: 17s - loss: 6.2564 - acc: 0.03 - ETA: 17s - loss: 6.2574 - acc: 0.03 - ETA: 17s - loss: 6.2524 - acc: 0.03 - ETA: 17s - loss: 6.2386 - acc: 0.03 - ETA: 17s - loss: 6.2263 - acc: 0.03 - ETA: 17s - loss: 6.2305 - acc: 0.03 - ETA: 17s - loss: 6.2273 - acc: 0.03 - ETA: 17s - loss: 6.2282 - acc: 0.03 - ETA: 16s - loss: 6.2324 - acc: 0.03 - ETA: 16s - loss: 6.2277 - acc: 0.03 - ETA

18063/18063 [==============================] - ETA: 17s - loss: 5.8321 - acc: 0.04 - ETA: 21s - loss: 5.8998 - acc: 0.07 - ETA: 23s - loss: 5.8784 - acc: 0.07 - ETA: 22s - loss: 5.9819 - acc: 0.06 - ETA: 22s - loss: 6.0155 - acc: 0.06 - ETA: 22s - loss: 6.0094 - acc: 0.05 - ETA: 22s - loss: 6.0293 - acc: 0.05 - ETA: 21s - loss: 6.0538 - acc: 0.05 - ETA: 21s - loss: 6.0773 - acc: 0.04 - ETA: 20s - loss: 6.0954 - acc: 0.05 - ETA: 20s - loss: 6.0968 - acc: 0.04 - ETA: 19s - loss: 6.0917 - acc: 0.04 - ETA: 19s - loss: 6.1186 - acc: 0.04 - ETA: 19s - loss: 6.1116 - acc: 0.04 - ETA: 18s - loss: 6.1097 - acc: 0.04 - ETA: 18s - loss: 6.0963 - acc: 0.05 - ETA: 18s - loss: 6.1162 - acc: 0.04 - ETA: 18s - loss: 6.1085 - acc: 0.04 - ETA: 18s - loss: 6.1155 - acc: 0.04 - ETA: 19s - loss: 6.1101 - acc: 0.04 - ETA: 18s - loss: 6.1001 - acc: 0.04 - ETA: 18s - loss: 6.0928 - acc: 0.04 - ETA: 18s - loss: 6.0883 - acc: 0.04 - ETA: 18s - loss: 6.1000 - acc: 0.04 - ETA: 19s - loss: 6.1000 - acc: 0.04 - ETA

...

18063/18063 [==============================] - ETA: 16s - loss: 0.9879 - acc: 0.77 - ETA: 21s - loss: 1.0122 - acc: 0.76 - ETA: 20s - loss: 1.0511 - acc: 0.76 - ETA: 20s - loss: 1.0640 - acc: 0.75 - ETA: 20s - loss: 1.1044 - acc: 0.75 - ETA: 19s - loss: 1.0639 - acc: 0.76 - ETA: 19s - loss: 1.0570 - acc: 0.75 - ETA: 19s - loss: 1.0701 - acc: 0.75 - ETA: 19s - loss: 1.0889 - acc: 0.74 - ETA: 18s - loss: 1.0768 - acc: 0.74 - ETA: 19s - loss: 1.0689 - acc: 0.74 - ETA: 18s - loss: 1.0750 - acc: 0.74 - ETA: 18s - loss: 1.0940 - acc: 0.74 - ETA: 18s - loss: 1.0931 - acc: 0.73 - ETA: 18s - loss: 1.1021 - acc: 0.73 - ETA: 17s - loss: 1.0895 - acc: 0.73 - ETA: 17s - loss: 1.0951 - acc: 0.73 - ETA: 17s - loss: 1.0999 - acc: 0.73 - ETA: 17s - loss: 1.1052 - acc: 0.73 - ETA: 17s - loss: 1.1081 - acc: 0.73 - ETA: 17s - loss: 1.0994 - acc: 0.73 - ETA: 17s - loss: 1.0982 - acc: 0.73 - ETA: 16s - loss: 1.0984 - acc: 0.73 - ETA: 16s - loss: 1.1008 - acc: 0.73 - ETA: 16s - loss: 1.0980 - acc: 0.73 - ETA

18063/18063 [==============================] - ETA: 18s - loss: 1.1925 - acc: 0.75 - ETA: 18s - loss: 1.1431 - acc: 0.73 - ETA: 17s - loss: 1.1718 - acc: 0.72 - ETA: 18s - loss: 1.1608 - acc: 0.73 - ETA: 18s - loss: 1.1790 - acc: 0.71 - ETA: 17s - loss: 1.1845 - acc: 0.71 - ETA: 18s - loss: 1.1382 - acc: 0.72 - ETA: 18s - loss: 1.1400 - acc: 0.72 - ETA: 17s - loss: 1.1244 - acc: 0.72 - ETA: 17s - loss: 1.1254 - acc: 0.72 - ETA: 17s - loss: 1.1332 - acc: 0.72 - ETA: 17s - loss: 1.1473 - acc: 0.72 - ETA: 17s - loss: 1.1192 - acc: 0.73 - ETA: 17s - loss: 1.1438 - acc: 0.72 - ETA: 17s - loss: 1.1353 - acc: 0.72 - ETA: 17s - loss: 1.1390 - acc: 0.72 - ETA: 16s - loss: 1.1370 - acc: 0.72 - ETA: 16s - loss: 1.1360 - acc: 0.72 - ETA: 16s - loss: 1.1352 - acc: 0.72 - ETA: 16s - loss: 1.1269 - acc: 0.72 - ETA: 16s - loss: 1.1270 - acc: 0.72 - ETA: 16s - loss: 1.1259 - acc: 0.73 - ETA: 16s - loss: 1.1286 - acc: 0.72 - ETA: 16s - loss: 1.1397 - acc: 0.72 - ETA: 16s - loss: 1.1325 - acc: 0.72 - ETA

18063/18063 [==============================] - ETA: 16s - loss: 1.0918 - acc: 0.74 - ETA: 15s - loss: 1.0453 - acc: 0.74 - ETA: 16s - loss: 1.0153 - acc: 0.75 - ETA: 16s - loss: 1.0173 - acc: 0.75 - ETA: 16s - loss: 1.0754 - acc: 0.74 - ETA: 17s - loss: 1.1117 - acc: 0.73 - ETA: 16s - loss: 1.1416 - acc: 0.72 - ETA: 16s - loss: 1.1407 - acc: 0.72 - ETA: 16s - loss: 1.1341 - acc: 0.72 - ETA: 16s - loss: 1.1539 - acc: 0.72 - ETA: 16s - loss: 1.1334 - acc: 0.72 - ETA: 15s - loss: 1.1446 - acc: 0.72 - ETA: 15s - loss: 1.1546 - acc: 0.72 - ETA: 15s - loss: 1.1475 - acc: 0.72 - ETA: 15s - loss: 1.1778 - acc: 0.71 - ETA: 15s - loss: 1.1828 - acc: 0.71 - ETA: 15s - loss: 1.1857 - acc: 0.71 - ETA: 15s - loss: 1.1806 - acc: 0.71 - ETA: 15s - loss: 1.1959 - acc: 0.70 - ETA: 15s - loss: 1.1940 - acc: 0.70 - ETA: 15s - loss: 1.1898 - acc: 0.70 - ETA: 15s - loss: 1.1928 - acc: 0.70 - ETA: 15s - loss: 1.1884 - acc: 0.70 - ETA: 15s - loss: 1.1985 - acc: 0.70 - ETA: 15s - loss: 1.1998 - acc: 0.70 - ETA

18063/18063 [==============================] - ETA: 16s - loss: 1.1802 - acc: 0.73 - ETA: 19s - loss: 1.0601 - acc: 0.76 - ETA: 18s - loss: 1.0900 - acc: 0.75 - ETA: 18s - loss: 1.1508 - acc: 0.73 - ETA: 18s - loss: 1.1689 - acc: 0.72 - ETA: 17s - loss: 1.2019 - acc: 0.71 - ETA: 17s - loss: 1.2003 - acc: 0.71 - ETA: 17s - loss: 1.1964 - acc: 0.71 - ETA: 17s - loss: 1.1758 - acc: 0.72 - ETA: 16s - loss: 1.1934 - acc: 0.72 - ETA: 16s - loss: 1.1900 - acc: 0.72 - ETA: 16s - loss: 1.1872 - acc: 0.71 - ETA: 16s - loss: 1.1739 - acc: 0.72 - ETA: 16s - loss: 1.1702 - acc: 0.72 - ETA: 15s - loss: 1.1622 - acc: 0.72 - ETA: 15s - loss: 1.1536 - acc: 0.72 - ETA: 15s - loss: 1.1419 - acc: 0.72 - ETA: 15s - loss: 1.1341 - acc: 0.72 - ETA: 15s - loss: 1.1294 - acc: 0.72 - ETA: 15s - loss: 1.1400 - acc: 0.72 - ETA: 15s - loss: 1.1475 - acc: 0.72 - ETA: 15s - loss: 1.1515 - acc: 0.72 - ETA: 14s - loss: 1.1506 - acc: 0.72 - ETA: 14s - loss: 1.1358 - acc: 0.72 - ETA: 14s - loss: 1.1454 - acc: 0.72 - ETA

18063/18063 [==============================] - ETA: 17s - loss: 0.9665 - acc: 0.75 - ETA: 18s - loss: 1.1069 - acc: 0.74 - ETA: 17s - loss: 1.1300 - acc: 0.73 - ETA: 18s - loss: 1.0830 - acc: 0.73 - ETA: 18s - loss: 1.0813 - acc: 0.73 - ETA: 18s - loss: 1.1070 - acc: 0.72 - ETA: 18s - loss: 1.0816 - acc: 0.73 - ETA: 18s - loss: 1.1041 - acc: 0.73 - ETA: 17s - loss: 1.0733 - acc: 0.73 - ETA: 17s - loss: 1.0666 - acc: 0.73 - ETA: 17s - loss: 1.0782 - acc: 0.73 - ETA: 17s - loss: 1.0750 - acc: 0.73 - ETA: 17s - loss: 1.0839 - acc: 0.73 - ETA: 17s - loss: 1.0772 - acc: 0.73 - ETA: 17s - loss: 1.0779 - acc: 0.73 - ETA: 17s - loss: 1.0759 - acc: 0.73 - ETA: 17s - loss: 1.0813 - acc: 0.73 - ETA: 17s - loss: 1.0805 - acc: 0.73 - ETA: 17s - loss: 1.0854 - acc: 0.72 - ETA: 16s - loss: 1.0843 - acc: 0.72 - ETA: 16s - loss: 1.0862 - acc: 0.72 - ETA: 16s - loss: 1.0928 - acc: 0.72 - ETA: 16s - loss: 1.0918 - acc: 0.72 - ETA: 16s - loss: 1.0916 - acc: 0.72 - ETA: 16s - loss: 1.0961 - acc: 0.72 - ETA

In [17]:
# Saving the model and the tokenizer
model.save('macbeth_model.h5')
dump(tokenizer,open('my_macbeth_tokenizer','wb'))

## Predict the next words

In [18]:
# using random sequences
random.seed(101)
random_pick = random.randint(0,len(text_sequences))
random_seed_text = text_sequences[random_pick]
seed_text = ' '.join(random_seed_text)

In [19]:
# generate the next word
model = load_model('macbeth_model.h5')
tokenizer = load(open('my_macbeth_tokenizer','rb'))
generator_text(model, tokenizer,seq_len, seed_text=seed_text, num_gen_words=25)

"know it further before where they vanish'd you in no lesse materiall in me naught in the winde macb i without your sad bosomes empty"